In [ ]:
from sepal_ui import sepalwidgets as sw
import ipyvuetify as v
from functools import partial

from utils import messages as ms
from scripts import run

In [ ]:
#use a class to define your input and output in order to have mutable variables
class Mspa_process_io:
    def __init__(self):
        #set up your inputs
        self.foreground_connectivity = None
        self.edge_width = None
        self.transition_core = True
        self.separate_feature = True
        self.statistics = True
        
mspa_process_io = Mspa_process_io()

In [ ]:
#create an empty result tile that will be filled with displayable plot, map, links, text
id_ = "mspa_process_widget"
title = "Results"
mspa_results = sw.Tile(id_, title, inputs=['No results to display yet'])

In [ ]:
#create the output alert 
mspa_process_output = sw.Alert()#.add_msg(ms.MSPA_MESSAGE)

#create the button that start your process
mspa_process_btn = sw.Btn('Run mspa analysis')

#create the widgets following ipyvuetify requirements 
mspa_connectivity = v.Select(
    items   = [4, 8], 
    label   = 'Foreground connectivity', 
    v_model = None
)
mspa_edge_slider = v.Slider(
    label       = 'Edge width (m)',
    max         = 300,
    class_      = "mt-5", 
    thumb_label = 'always', 
    step        = 30,
    v_model     = 0
)
mspa_transition = v.Switch(label="Transition Core - Loop/Bridge", v_model=True)
mspa_feature = v.Switch(label="Separate internal from external features", v_model=True)
mspa_statistics = v.Switch(label="Compute statistics", v_model=True)


mspa_process_inputs = [
    mspa_connectivity, 
    mspa_edge_slider, 
    mspa_transition, 
    mspa_feature, 
    mspa_statistics
]

#bind the widget to the inputs
mspa_process_output.bind(mspa_connectivity, mspa_process_io, 'foreground_connectivity')
mspa_process_output.bind(mspa_edge_slider, mspa_process_io, 'edge_width')
mspa_process_output.bind(mspa_transition, mspa_process_io, 'transition_core')
mspa_process_output.bind(mspa_feature, mspa_process_io, 'separate_feature')
mspa_process_output.bind(mspa_statistics, mspa_process_io, 'statistics')

#create a process tile
id_ = "mspa_process_widget"
title = 'Create fragmentation map'

mspa_process = sw.Tile(
    id_, 
    title, 
    btn=mspa_process_btn, 
    inputs=mspa_process_inputs, 
    output=mspa_process_output
)

In [ ]:
#bind the button to the process by writing a custom function
def process_start(widget, event, data, output, mspa_results):    
    
    #toggle the loading button
    widget.toggle_loading()
    
    #try:
    #load inputs
    #asset = getattr(gfc_aoi_io, 'assetId')
    threshold = getattr(gfc_viz_io, 'threshold')
    proj_map = getattr(gfc_export_io, 'proj_map')
    foreground_connectivity = getattr(mspa_process_io, 'foreground_connectivity')
    edge_width = getattr(mspa_process_io,'edge_width')
    transition_core = getattr(mspa_process_io, 'transition_core')
    separate_feature = getattr(mspa_process_io, 'separate_feature')
    statistics = getattr(mspa_process_io, 'statistics')
    
    #check inputs 
    if not output.check_input(gfc_aoi_io.get_aoi_ee(), ms.NO_AOI): return widget.toggle_loading()
    if not output.check_input(proj_map, ms.NO_MAP): return widget.toggle_loading()
    if not output.check_input(foreground_connectivity, ms.NO_INPUT): return widget.toggle_loading()
    if not output.check_input(edge_width, ms.NO_INPUT): return widget.toggle_loading()
    
    #launch any process you want
    output_layout = run.mspaAnalysis(
        clip_map=proj_map, 
        aoi_io=gfc_aoi_io, 
        threshold=threshold, 
        foreground_connectivity=foreground_connectivity, 
        edge_width=int(edge_width/30), #to reduce to pixel width, 
        transition_core=transition_core, 
        separate_feature=separate_feature, 
        statistics=statistics,
        output=output
    )
    
    
    #display the image 
    mspa_results.set_content(output_layout)
    
    #except Exception as e: 
    #    output.add_live_msg(str(e), 'error')
        
    #toggle the loading button
    widget.toggle_loading()
    
    return

mspa_process_btn.on_event('click', partial(
    process_start, 
    output=mspa_process_output,
    mspa_results=mspa_results
))

In [ ]:
#this tiles will only be displayed if you launch voila from this file 
mspa_process

In [ ]:
#this tiles will only be displayed if you launch voila from this file 
mspa_results